In [1]:
#This script identifies the epitope and therefore identifies the epitope - paratope complex.
#@author: Henriette Capel
#@date: 02 March 2022

In [ ]:
#Import modules
import Bio.PDB
from Bio.Data.IUPACData import protein_letters_3to1
from Bio.PDB.Polypeptide import standard_aa_names

from ABDB import database as db
import pandas as pd

%load_ext nb_black

In [3]:
#Set numbering scheme to IMGT
db.set_numbering_scheme('imgt')

<IPython.core.display.Javascript object>

In [4]:
#Constants
DISTANCE_CUTOFF = 4.5
columns_dataframe = ['pdb', 'antigen_chain', 'antibody_chain', 'antigen_sequence', 'antibody_sequence', 'cdr_sequence', 
                           'epitope_binary_MCDR', 'epitope_MCDR', 'epitope_MCDR_dict', 'paratope_binary_MCDR', 'paratope_MCDR', 'paratope_MCDR_dict',
                           'epitope_binary_MWV', 'epitope_MWV', 'epitope_MWV_dict','paratope_binary_MWV', 'paratope_MWV', 'paratope_MWV_dict']

<IPython.core.display.Javascript object>

In [5]:
#Functions 
def store_ag_ab_cdr_atoms(pdb_id, ab_type): #store_CDRatoms_antigenatoms
    #This functions indicate the residues of the CDR, full antibody and the full antigen. 
    #output: three dictionaries. dict_chain_antigenresidues contains the ag chain and the corresponding residues, 
    # dict_chain_antibodyresidues contains the ab chain and corresponding residues, and dict_chain_targetresidues
    # containing the ab chain and the residues belonging to the CDR. The residues are stored in the Bio PDB format.

    p = db.fetch(pdb_id)
    imgt_structure = p.get_structure(scheme="imgt", definition = 'imgt')
    fabs_of_interest = chains_of_interest(p, ab_type)
    fab_target_dict = create_target_dict(p,fabs_of_interest)
    
    dict_chain_antigenresidues = {}
    dict_chain_antibodyresidues = {}
    dict_chain_targetresidues = {}

    for model in imgt_structure:
        model_id = model.get_id()
        for chain_type in model:
            chain_type_id = chain_type.get_id()
            
            if chain_type_id == "Antigen":
                for chain in chain_type:
                    chain_id = chain.get_id()
                    residues_antigen = get_all_residues(chain)
                    if len(residues_antigen) != 0: #Seen that antigen does not contain any residues. So only store if contains residues
                        dict_chain_antigenresidues[chain_id] = residues_antigen
            
            else: #dealing with antibody
                for chain in chain_type:
                    chain_id = chain.get_id()
                    if chain_id in fabs_of_interest: 

                        #Store residues of whole antibody
                        residues_antibody = get_all_residues(chain)     
                        dict_chain_antibodyresidues[chain_id] = residues_antibody

                        #Store only residues of CDR
                        list_targetresidues = []
                        target_dict = fab_target_dict[chain_id]
                        for num_letter in target_dict.keys():
                            target_num = num_letter[0]
                            target_letter = num_letter[1]
                            try: 
                                target_selected = chain[" ", target_num, target_letter] #Store in Bio PDB format
                                list_targetresidues.append(target_selected)
                            except: 
                                print(f"target does not exist: pdb: {pdb_id}, chain: {chain_id}, residue: ( , {target_num}, {target_letter})")
                                
                        dict_chain_targetresidues[chain_id] = list_targetresidues
                    
    return dict_chain_antigenresidues, dict_chain_antibodyresidues, dict_chain_targetresidues

def create_target_dict(pdb_details, fabs_interest):
    #Indicates the CDR sequence and the corresponding position. 
    #Output: a dictionary containing the chain of the fab (in case of the full lenght antibody the chain of the heavy 
    #and light chain seperately) and they target residue in the residue Bio PDB format.
    
    fab_target_dict = {}
    for fab in pdb_details.get_fabs():
        fab_chain_id = str(fab).split("\n")[0].split(" ")[1]#construct chain out of fab
        if fab_chain_id in fabs_interest:
            seq_imgt_dict = fab.get_CDR_sequences(definition = 'imgt', scheme = 'imgt')
           
            if len(seq_imgt_dict.keys()) == 6: #full length antibody 
                CDRH_dict = {}
                CDRL_dict = {}
                CDRheavy = ['CDRH1', 'CDRH2', 'CDRH3']
                for CDR_type in seq_imgt_dict.keys():
                    if CDR_type in CDRheavy:
                        CDRH_dict[CDR_type] = seq_imgt_dict[CDR_type]
                    else:
                        CDRL_dict[CDR_type] = seq_imgt_dict[CDR_type]
                list_cdr_dicts = [CDRH_dict, CDRL_dict]
                for i in range(len(list_cdr_dicts)):
                    chain_id = fab_chain_id[i]
                    target_dict = translate_residue_to_targetid(list_cdr_dicts[i])
                    fab_target_dict[chain_id] = target_dict
            else: #single domain antibody
                target_dict = translate_residue_to_targetid(seq_imgt_dict)
                fab_target_dict[fab_chain_id] = target_dict
    return fab_target_dict

def translate_residue_to_targetid(list_cdrs_seq):
    #translate the residue format from fab.get_sequence into the Bio PDB format
    target_dict = {}
    for CDR_type in list_cdrs_seq:
        for element in list_cdrs_seq[CDR_type]:
            if element[0][1] == " ": #no insertions
                key_num = element[0][0]
                target_dict[(key_num, " ")] = element[1]
            else:
                key_num = element[0][0]
                key_letter = element[0][1]
                target_dict[(key_num, key_letter)] = element[1]
    return target_dict

def determine_interactingresidues(dict_antigenresidues, dict_antibodyresidues, dict_targetresidues, pdb_id, distance_cutoff):
    #Check if at least one residue in CDR is in contact
    #If this is the case stores the epitope and paratope based on a distance cut off given by distance_cutoff.
    #Method 1: epitope determination based on in contact with only CDR region
    #Method 2: epitope determination based on in contact with any antibody variable domain residue
    #Output is a dictionary containing the antigen chain, antibody chain, epitope residues and paratope residues 
    #(for both methods)
    
    dict_antigen_antibody_epi_para = {}

    for antigen_chain in dict_antigenresidues.keys():
        dict_antibody_epi_para = {}
        
        atoms_antigen = Bio.PDB.Selection.unfold_entities(dict_antigenresidues[antigen_chain], 'A') 
        ns = Bio.PDB.NeighborSearch(atoms_antigen)

        for antibody_chain in dict_targetresidues.keys():
            #Method 1 of epitope determination
            list_CDRresidues = dict_targetresidues[antibody_chain]
            epitope_residues, paratope_residues, list_number_binding_CDR = determine_epi_para(list_CDRresidues, ns, distance_cutoff)

            #Method 2 of epitope determination
            list_antibody_residues = dict_antibodyresidues[antibody_chain]
            epitope2_residues, paratope2_residues, list_number_binding_WV = determine_epi_para(list_antibody_residues, ns, distance_cutoff)

            #Filter out antibody chains that are not in contact with antigen.
            if sum(list_number_binding_CDR)>0: 
                dict_epi_para = {'epitope': epitope_residues,
                                'paratope': paratope_residues,
                                'epitope_2': epitope2_residues,
                                'paratope_2': paratope2_residues}
                dict_antibody_epi_para[antibody_chain] = dict_epi_para

        dict_antigen_antibody_epi_para[antigen_chain] = dict_antibody_epi_para

    return dict_antigen_antibody_epi_para

def determine_epi_para(residue_list, ns, distance_cutoff):
    #Determines the epitope and paratope residues
    #Input should be a list containing Bio PDB residue objects
    
    epitope_residues = set()
    paratope_residues = set()
    
    list_number_binding = []
    for residue in residue_list:
        residue_count = 0 
        for atom in residue:
            close_atoms = ns.search(atom.coord, distance_cutoff, level="R")
            epitope_residues.update(close_atoms)
            if len(close_atoms) > 0:
                residue_count += 1

        #Check if selected residue is in close contact. If so store as paratope residue
        if residue_count > 0:
            paratope_residues.add(residue)

        list_number_binding.append(residue_count)
    
    return epitope_residues, paratope_residues, list_number_binding

def create_store_annoations(pdb_id, ab_type, dict_antigenresidues, dict_antibodyresidues, dict_antigen_antibody_epi_para):
    #Create the epitope and paratope annotations using the make_seq_annotation function
    #Dertermines the CDR sequences using the function get_cdr_sequence
    #Stores the data per interacting antigen and antibody chain in a dictionary. 
    #The output is a list containing these dictionaries, so all interactions between antigen and antibody chain per pdb entry.
    
    if dict_antigenresidues.keys() != dict_antigen_antibody_epi_para.keys():
        print("WARNING: antigen chains do not correspond")
    
    data_list = []
    
    for ag_chain in dict_antigenresidues.keys(): 
        for ab_chain in dict_antigen_antibody_epi_para[ag_chain].keys():
            #Make annotations
            seq_ag, epitope_anno_binary, epitope_anno = make_seq_annotation(dict_antigenresidues[ag_chain], dict_antigen_antibody_epi_para[ag_chain][ab_chain]['epitope'])
            seq_ab, paratope_anno_binary, paratope_anno = make_seq_annotation(dict_antibodyresidues[ab_chain], dict_antigen_antibody_epi_para[ag_chain][ab_chain]['paratope'])
            seq_ag2, epitope2_anno_binary, epitope2_anno = make_seq_annotation(dict_antigenresidues[ag_chain], dict_antigen_antibody_epi_para[ag_chain][ab_chain]['epitope_2'])
            seq_ab2, paratope2_anno_binary, paratope2_anno = make_seq_annotation(dict_antibodyresidues[ab_chain], dict_antigen_antibody_epi_para[ag_chain][ab_chain]['paratope_2'])
            
            epitope_MCDR_dict = create_dict_interaction_residues(dict_antigen_antibody_epi_para[ag_chain][ab_chain]['epitope'])
            epitope_MWV_dict = create_dict_interaction_residues(dict_antigen_antibody_epi_para[ag_chain][ab_chain]['epitope_2'])
            paratope_MCDR_dict = create_dict_interaction_residues(dict_antigen_antibody_epi_para[ag_chain][ab_chain]['paratope'])
            paratope_MWV_dict = create_dict_interaction_residues(dict_antigen_antibody_epi_para[ag_chain][ab_chain]['paratope_2'])
            
            #Determine CDR sequence
            #Note the heavy and light chain of the full lenght antibody will both have have all the cdr_types and sequences
            dict_cdr_seq = get_cdr_sequence(pdb_id, ab_type, ab_chain)
            
            #add CDR sequence?
            dict_data = {'pdb': pdb_id,
                         'antigen_chain': ag_chain,
                         'antibody_chain': ab_chain, 
                         'antigen_sequence': seq_ag,
                         'antibody_sequence': seq_ab,
                         'cdr_sequence': dict_cdr_seq,
                         'epitope_binary_MCDR': epitope_anno_binary, 
                         'epitope_MCDR': epitope_anno,
                         'epitope_MCDR_dict': epitope_MCDR_dict,
                         'paratope_binary_MCDR': paratope_anno_binary, 
                         'paratope_MCDR': paratope_anno,
                         'paratope_MCDR_dict': paratope_MCDR_dict,
                         'epitope_binary_MWV': epitope2_anno_binary, 
                         'epitope_MWV': epitope2_anno,
                         'epitope_MWV_dict': epitope_MWV_dict,
                         'paratope_binary_MWV': paratope2_anno_binary, 
                         'paratope_MWV': paratope2_anno,
                         'paratope_MWV_dict': paratope_MWV_dict}
            
            data_list.append(dict_data)
                
    return data_list

def create_dict_interaction_residues(residue_list):
    #Store the residues, also insertion letter!
    interacting_residue_position_aa_dict = {}
    for residue in residue_list:
        res_name = residue.resname.strip()
        res_name_3code = res_name[0] + res_name[1:].lower()
        res_name_1code = Bio.Data.IUPACData.protein_letters_3to1[res_name_3code]
        res_position = residue.get_id()[1]
        res_insertionletter = residue.get_id()[2]
        if res_insertionletter != " ": #Save insertion residues
            res_position = str(res_position) + "_" + res_insertionletter
        interacting_residue_position_aa_dict[res_position]=res_name_1code
    return interacting_residue_position_aa_dict
    
def get_cdr_sequence(pdb_id, ab_type, ab_chain):
    p = db.fetch(pdb_id)
    for fab in p.get_fabs():
        fab_chain = str(fab).split("\n")[0].split(" ")[1]
        if fab_chain == ab_chain or ab_chain in fab_chain: 
            dict_cdr = construct_cdr_seq(fab, ab_type)
    return dict_cdr
    
def construct_cdr_seq(fab, ab_type):
    #Determines the CDR sequence and correcly formats the dictionary (See CDR_unique_sequences script)
    seq_imgt = fab.get_CDR_sequences(definition = 'imgt', scheme = 'imgt')
    dict_CDR_seq = {}
    concate_seq = "" 
    for CDR_type in seq_imgt.keys():
        seq = "" 
        for element in seq_imgt.get(CDR_type):
            seq += element[1]
        dict_CDR_seq[CDR_type] = seq    
        
        if ab_type == "Fv" and CDR_type == "CDRL1": #Store the length of the concatenated CDRH domains such that can store CDRH and CDRL seperately later on
            length_CDRH = len(concate_seq)
    
        concate_seq += seq
        
    if ab_type == "Nb":
        dict_CDR_seq["CDRH"] = concate_seq #Store CDRH
    if ab_type == "VL":
        dict_CDR_seq["CDRL"] = concate_seq #Store CDRL
    if ab_type == "Fv":
        dict_CDR_seq["CDRH"] = concate_seq[:length_CDRH] #Store concatenated CDRH 
        dict_CDR_seq["CDRL"] = concate_seq[length_CDRH: ]#Store CDRL
        dict_CDR_seq["CDRHL"] = concate_seq #Store CDRHL
        
    return dict_CDR_seq

def combine_HLchains(data_list, pdb_id):
    #Only needed for full length (Fv) antibodies
    data_fv = []
    
    p = db.fetch(pdb_id)
    fabs_of_interest = chains_of_interest(p, "Fv")
    fabs_combined_interest = [item for item in fabs_of_interest if len(item)==2] 
    for element in fabs_combined_interest: #Element = heavy and light chain connected
        heavy_chain = element[0]
        light_chain = element[1]
    
        for i in range(len(data_list)):
            if data_list[i]["antibody_chain"] == heavy_chain:
                dict_heavy_info = data_list[i]
            elif data_list[i]["antibody_chain"] == light_chain:
                dict_light_info = data_list[i]
        
        #Note: if antibody not interacting than these chains can not find in the data_list
        try:
            #Note: the heavy and light chain can bind to different antigens. Do not included them (for now)
            if dict_heavy_info["antigen_chain"] == dict_light_info["antigen_chain"]:
                #Combine the epitope annotations
                cb_epitope_binary_MCDR = combine_epitopes(dict_heavy_info['epitope_binary_MCDR'], dict_light_info['epitope_binary_MCDR'])
                cb_epitope_MCDR = combine_epitopes(dict_heavy_info['epitope_MCDR'], dict_light_info['epitope_MCDR'])
                cb_epitope_binary_MWV = combine_epitopes(dict_heavy_info['epitope_binary_MWV'], dict_light_info['epitope_binary_MWV'])
                cb_epitope_MWV = combine_epitopes(dict_heavy_info['epitope_MWV'], dict_light_info['epitope_MWV'])
                                                    
                epitope_MCDR_dict = {heavy_chain: dict_heavy_info['epitope_MCDR_dict'],
                                     light_chain: dict_light_info['epitope_MCDR_dict'],
                                     element: dict_heavy_info['epitope_MCDR_dict'] | dict_light_info['epitope_MCDR_dict']}
                
                epitope_MWV_dict = {heavy_chain: dict_heavy_info['epitope_MWV_dict'],
                                    light_chain: dict_light_info['epitope_MWV_dict'],
                                    element: dict_heavy_info['epitope_MWV_dict'] | dict_light_info['epitope_MWV_dict']}
                                                    
                #Save the information
                dict_data_fv = {'pdb': dict_heavy_info['pdb'],
                                'antigen_chain': dict_heavy_info['antigen_chain'],
                                'antibody_chain': element,
                                'antigen_sequence': dict_heavy_info['antigen_sequence'],
                                'antibody_sequence': {heavy_chain: dict_heavy_info['antibody_sequence'], 
                                               light_chain: dict_light_info['antibody_sequence']},
                                'cdr_sequence': dict_heavy_info['cdr_sequence'], #Heavy and light contain same information for this feature
                                'epitope_binary_MCDR': {heavy_chain: dict_heavy_info['epitope_binary_MCDR'], 
                                                 light_chain: dict_light_info['epitope_binary_MCDR'], 
                                                 element: cb_epitope_binary_MCDR},
                                'epitope_MCDR': {heavy_chain: dict_heavy_info['epitope_MCDR'], 
                                          light_chain: dict_light_info['epitope_MCDR'], 
                                          element: cb_epitope_MCDR},
                                'epitope_MCDR_dict': epitope_MCDR_dict,
                                'paratope_binary_MCDR': {heavy_chain: dict_heavy_info['paratope_binary_MCDR'], 
                                                  light_chain: dict_light_info['paratope_binary_MCDR']}, 
                                'paratope_MCDR': {heavy_chain: dict_heavy_info['paratope_MCDR'], 
                                           light_chain: dict_light_info['paratope_MCDR']},
                                'paratope_MCDR_dict': {heavy_chain: dict_heavy_info['paratope_MCDR_dict'], 
                                                       light_chain: dict_light_info['paratope_MCDR_dict']},
                                'epitope_binary_MWV': {heavy_chain: dict_heavy_info['epitope_binary_MWV'], 
                                                light_chain: dict_light_info['epitope_binary_MWV'], 
                                                element: cb_epitope_binary_MWV}, 
                                'epitope_MWV': {heavy_chain: dict_heavy_info['epitope_MWV'], 
                                         light_chain: dict_light_info['epitope_MWV'], 
                                         element: cb_epitope_MWV},
                                'epitope_MWV_dict': epitope_MWV_dict,
                                'paratope_binary_MWV': {heavy_chain: dict_heavy_info['paratope_binary_MWV'], 
                                                 light_chain: dict_light_info['paratope_binary_MWV']},
                                'paratope_MWV': {heavy_chain: dict_heavy_info['paratope_MWV'], 
                                          light_chain: dict_light_info['paratope_MWV']},
                               'paratope_MWV_dict': {heavy_chain: dict_heavy_info['paratope_MWV_dict'], 
                                                       light_chain: dict_light_info['paratope_MWV_dict']}}

                data_fv.append(dict_data_fv)
        except NameError:
            pass
        
    return data_fv

def combine_epitopes(epitope_heavy, epitope_light):
    new_epitope = ""
    for i in range(len(epitope_heavy)):
        if epitope_heavy[i] == "-" or epitope_heavy[i] == "0":
            new_epitope += epitope_light[i]
        else:
            new_epitope += epitope_heavy[i]
        
    return new_epitope   

def chains_of_interest(pdb_details, ab_type):
    fabs_of_interest = []
    for fab in pdb_details.get_fabs():
        chain_id = str(fab).split("\n")[0].split(" ")[1]
        if ab_type == "Nb" and fab.VL == "NA": #select only the nanobody fragment
            fabs_of_interest.append(chain_id)
        elif ab_type == "VL" and fab.VH == "NA": #select VL-only fragments
            fabs_of_interest.append(chain_id)
        elif ab_type == "Fv" and fab.VL != "NA" and fab.VH != "NA" and fab.VH != fab.VL:
            fabs_of_interest.append(chain_id)
            #Add the VL and VH chain seperately
            fabs_of_interest.append(chain_id[0])
            fabs_of_interest.append(chain_id[1])
    return fabs_of_interest
    

def get_all_residues(chain):
    #Only store the standaard aa (so exclude HOH for example)
    #Input should be a BIO PDB chain object 
    residues_antigen = []
    for residue_ag in Bio.PDB.Selection.unfold_entities(chain, 'R'):
        res_ag_name = residue_ag.resname.strip()
        if res_ag_name in Bio.PDB.Polypeptide.standard_aa_names:
            residues_antigen.append(residue_ag)
    return residues_antigen

def make_seq_annotation(residue_list, list_interacting_residues):
    #Annotates the epitope and paratope both binary and with amino acids. 
    #Input is a list of the interacting residues in Bio PDB format
    seq = ""
    annotated_seq = ""
    annotated_binary = ""
    for residue in residue_list:
        res_name = residue.resname.strip()
        res_name_3code = res_name[0] + res_name[1:].lower()
        res_name_1code = Bio.Data.IUPACData.protein_letters_3to1[res_name_3code]
        seq += res_name_1code
        
        if residue in list_interacting_residues:
            annotated_seq += res_name_1code
            annotated_binary += "1"
        else:
            annotated_seq += "-"
            annotated_binary += "0"
            
    return seq, annotated_binary, annotated_seq

def store_in_dataframe(df, data_list):
    #Stores all dictionaries containing a ab-ag interaction in the data_list (per pdb) as a row in a pandas dataframe
    for data_dict in data_list:
        df_new_line = pd.DataFrame({k: [v] for k, v in data_dict.items()})
        df = pd.concat([df, df_new_line])
    return df

def remove_peptide_antigens(df):
    #Remove peptide antigen chains from the dataframe. These could be present because one PDB entry can have both a 
    #peptide and protein antigen in the structure. Or because SAbDab sometimes identify antigens as proteins although 
    #the size is between 30 and 50 residues. 
    new_df = df[df['antigen_sequence'].map(len) >= 50]
    
    return new_df

def main_(pdb_list, ab_type, distance_cutoff):
    #main function to determine, annotated and store all interactions between epitopes and paratopes in a dataset (pdb_list)
    #Input: list containing pdb entries, antibody type (Nb, VL, or Fv), and the distance cutoff 
    df = pd.DataFrame(columns=columns_dataframe)
    for pdb_item in pdb_list:
        dict_antigenresidues, dict_antibodyresidues, dict_targetresidues = store_ag_ab_cdr_atoms(pdb_item, ab_type)
        dict_antigen_antibody_epi_para = determine_interactingresidues(dict_antigenresidues, dict_antibodyresidues, dict_targetresidues, pdb_item, DISTANCE_CUTOFF)
        data_list = create_store_annoations(pdb_item, ab_type, dict_antigenresidues, dict_antibodyresidues, dict_antigen_antibody_epi_para)
        
        if ab_type == "Fv":
            data_list_fv = combine_HLchains(data_list, pdb_item)
            data_list = data_list_fv
        
        df = store_in_dataframe(df, data_list)
        
        #Still need to remove peptides from dataset
        df_without_peptides = remove_peptide_antigens(df)
    
    return df_without_peptides

<IPython.core.display.Javascript object>

In [6]:
#single domain antibodies
ab_type = "Nb"
with open(f"{ab_type}.txt") as file:
    list_Nbs = [line.rstrip() for line in file]
df_nb = main_(list_Nbs, ab_type, DISTANCE_CUTOFF)

target does not exist: pdb: 2p42, chain: B, residue: ( , 56,  )
target does not exist: pdb: 2p42, chain: D, residue: ( , 56,  )
target does not exist: pdb: 4ios, chain: D, residue: ( , 27,  )
target does not exist: pdb: 4ios, chain: D, residue: ( , 28,  )
target does not exist: pdb: 4ios, chain: D, residue: ( , 29,  )
target does not exist: pdb: 4ios, chain: D, residue: ( , 30,  )
target does not exist: pdb: 4ios, chain: E, residue: ( , 29,  )
target does not exist: pdb: 4ios, chain: E, residue: ( , 30,  )
target does not exist: pdb: 4ios, chain: E, residue: ( , 35,  )
target does not exist: pdb: 4ios, chain: F, residue: ( , 28,  )
target does not exist: pdb: 4ios, chain: F, residue: ( , 29,  )
target does not exist: pdb: 4ios, chain: F, residue: ( , 30,  )
target does not exist: pdb: 4ios, chain: F, residue: ( , 35,  )
target does not exist: pdb: 4ios, chain: G, residue: ( , 29,  )
target does not exist: pdb: 4ios, chain: G, residue: ( , 30,  )
target does not exist: pdb: 4ios, chain:

<IPython.core.display.Javascript object>

In [8]:
#full length antibodies
ab_type = "Fv"
with open(f"{ab_type}.txt") as file:
    list_Fv = [line.rstrip() for line in file]
df_fv = main_(list_Fv, "Fv", DISTANCE_CUTOFF)

target does not exist: pdb: 4f15, chain: B, residue: ( , 111,  )
target does not exist: pdb: 4f15, chain: B, residue: ( , 111, A)
target does not exist: pdb: 4f15, chain: B, residue: ( , 112, A)
target does not exist: pdb: 4f15, chain: B, residue: ( , 112,  )
target does not exist: pdb: 4f15, chain: B, residue: ( , 113,  )
target does not exist: pdb: 4f15, chain: B, residue: ( , 114,  )
target does not exist: pdb: 4f15, chain: B, residue: ( , 115,  )
target does not exist: pdb: 4f15, chain: B, residue: ( , 116,  )
target does not exist: pdb: 4f15, chain: C, residue: ( , 31,  )
target does not exist: pdb: 4f15, chain: C, residue: ( , 34,  )
target does not exist: pdb: 4f15, chain: C, residue: ( , 35,  )
target does not exist: pdb: 4f15, chain: C, residue: ( , 36,  )
target does not exist: pdb: 4f15, chain: E, residue: ( , 111,  )
target does not exist: pdb: 4f15, chain: E, residue: ( , 111, A)
target does not exist: pdb: 4f15, chain: E, residue: ( , 112, A)
target does not exist: pdb: 4

target does not exist: pdb: 6n5e, chain: G, residue: ( , 59,  )
target does not exist: pdb: 6n5e, chain: G, residue: ( , 62,  )
target does not exist: pdb: 6n5e, chain: G, residue: ( , 63,  )
target does not exist: pdb: 6n5e, chain: G, residue: ( , 64,  )
target does not exist: pdb: 3lhp, chain: H, residue: ( , 111, A)
target does not exist: pdb: 3lhp, chain: H, residue: ( , 111, B)
target does not exist: pdb: 3lhp, chain: H, residue: ( , 111, C)
target does not exist: pdb: 3lhp, chain: I, residue: ( , 111, A)
target does not exist: pdb: 3lhp, chain: I, residue: ( , 111, B)
target does not exist: pdb: 3lhp, chain: I, residue: ( , 111, C)
target does not exist: pdb: 6lxj, chain: F, residue: ( , 29,  )
target does not exist: pdb: 6lxj, chain: F, residue: ( , 30,  )
target does not exist: pdb: 6lxj, chain: F, residue: ( , 31,  )
target does not exist: pdb: 6lxj, chain: K, residue: ( , 29,  )
target does not exist: pdb: 6lxj, chain: K, residue: ( , 30,  )
target does not exist: pdb: 6lxj, 

target does not exist: pdb: 5b8c, chain: E, residue: ( , 27,  )
target does not exist: pdb: 7lf8, chain: H, residue: ( , 63,  )
target does not exist: pdb: 4hkx, chain: B, residue: ( , 56,  )
target does not exist: pdb: 4hkx, chain: B, residue: ( , 57,  )
target does not exist: pdb: 4hkx, chain: B, residue: ( , 65,  )
target does not exist: pdb: 7kzb, chain: A, residue: ( , 27,  )
target does not exist: pdb: 7kzb, chain: A, residue: ( , 28,  )
target does not exist: pdb: 7kzb, chain: A, residue: ( , 29,  )
target does not exist: pdb: 7kzb, chain: A, residue: ( , 65,  )
target does not exist: pdb: 5b3j, chain: H, residue: ( , 27,  )
target does not exist: pdb: 5b3j, chain: H, residue: ( , 28,  )
target does not exist: pdb: 6d2p, chain: L, residue: ( , 36,  )
target does not exist: pdb: 6j15, chain: H, residue: ( , 27,  )
target does not exist: pdb: 6j15, chain: H, residue: ( , 28,  )
target does not exist: pdb: 6j15, chain: H, residue: ( , 29,  )
target does not exist: pdb: 6j15, chain:

target does not exist: pdb: 2xqb, chain: L, residue: ( , 28,  )
target does not exist: pdb: 2xqb, chain: L, residue: ( , 29,  )
target does not exist: pdb: 2xqb, chain: L, residue: ( , 30,  )
target does not exist: pdb: 2xqb, chain: L, residue: ( , 35,  )
target does not exist: pdb: 2xqb, chain: L, residue: ( , 36,  )
target does not exist: pdb: 4dqo, chain: H, residue: ( , 111, H)
target does not exist: pdb: 7coe, chain: H, residue: ( , 111, D)
target does not exist: pdb: 7coe, chain: H, residue: ( , 111, E)
target does not exist: pdb: 7coe, chain: B, residue: ( , 111, B)
target does not exist: pdb: 7coe, chain: B, residue: ( , 111, C)
target does not exist: pdb: 7coe, chain: B, residue: ( , 111, D)
target does not exist: pdb: 7e7y, chain: C, residue: ( , 111, C)
target does not exist: pdb: 7ket, chain: B, residue: ( , 27,  )
target does not exist: pdb: 7ket, chain: B, residue: ( , 28,  )
target does not exist: pdb: 7ket, chain: B, residue: ( , 29,  )
target does not exist: pdb: 7ket,

target does not exist: pdb: 6bkb, chain: H, residue: ( , 27,  )
target does not exist: pdb: 6bkb, chain: H, residue: ( , 28,  )
target does not exist: pdb: 7lf7, chain: H, residue: ( , 37,  )
target does not exist: pdb: 5if0, chain: B, residue: ( , 28,  )
target does not exist: pdb: 5if0, chain: B, residue: ( , 29,  )
target does not exist: pdb: 5if0, chain: B, residue: ( , 30,  )
target does not exist: pdb: 5if0, chain: B, residue: ( , 31,  )
target does not exist: pdb: 5if0, chain: B, residue: ( , 32,  )
target does not exist: pdb: 5if0, chain: B, residue: ( , 33,  )
target does not exist: pdb: 5if0, chain: B, residue: ( , 34,  )
target does not exist: pdb: 5if0, chain: B, residue: ( , 35,  )
target does not exist: pdb: 5if0, chain: L, residue: ( , 28,  )
target does not exist: pdb: 5if0, chain: L, residue: ( , 29,  )
target does not exist: pdb: 5if0, chain: L, residue: ( , 30,  )
target does not exist: pdb: 5if0, chain: L, residue: ( , 31,  )
target does not exist: pdb: 5if0, chain:

target does not exist: pdb: 6uyf, chain: H, residue: ( , 27,  )
target does not exist: pdb: 6b9j, chain: L, residue: ( , 56,  )
target does not exist: pdb: 6b9j, chain: L, residue: ( , 57,  )
target does not exist: pdb: 6b9j, chain: L, residue: ( , 109,  )
target does not exist: pdb: 6b9j, chain: L, residue: ( , 113,  )
target does not exist: pdb: 6b9j, chain: L, residue: ( , 114,  )
target does not exist: pdb: 3u2s, chain: H, residue: ( , 111, H)
target does not exist: pdb: 3u2s, chain: H, residue: ( , 112, I)
target does not exist: pdb: 3u2s, chain: A, residue: ( , 111, H)
target does not exist: pdb: 3u2s, chain: A, residue: ( , 112, I)
target does not exist: pdb: 4lsr, chain: H, residue: ( , 31,  )
target does not exist: pdb: 4lsr, chain: H, residue: ( , 32,  )
target does not exist: pdb: 4lsr, chain: H, residue: ( , 32, A)
target does not exist: pdb: 4lsr, chain: H, residue: ( , 32, B)
target does not exist: pdb: 4lsr, chain: H, residue: ( , 33, C)
target does not exist: pdb: 4lsr,

target does not exist: pdb: 5d70, chain: L, residue: ( , 56,  )
target does not exist: pdb: 5d70, chain: L, residue: ( , 65,  )
target does not exist: pdb: 5d71, chain: L, residue: ( , 56,  )
target does not exist: pdb: 5d71, chain: L, residue: ( , 65,  )


<IPython.core.display.Javascript object>

In [11]:
##SAVE: write to file
# df_nb.to_csv("Nb_epitope_paratope.csv")
# df_fv.to_csv("Fv_epitope_paratope.csv")

<IPython.core.display.Javascript object>